In [4]:
import sys
import os
import pandas as pd
import seaborn as sns

# Add the path to the sys.path
path = '/home/lexi/deep_weather/deep_weather/'
if path not in sys.path:
    sys.path.append(path)

from datetime import datetime, timedelta
import time
import numpy as np
import matplotlib.pyplot as plt



In [5]:
import numpy as np
from scipy.ndimage import uniform_filter

def fss_init(threshold, scale):
    """Initialize a fractions skill score (FSS) verification object."""
    fss = dict(threshold=threshold, scale=scale, sum_fct_sq=0.0, sum_fct_obs=0.0, sum_obs_sq=0.0)
    return fss

def calculate_bp(data, threshold):
    """
    Calculate the Binary Predictor (BP) for the given data and threshold.
    
    Parameters:
    data (np.ndarray): Array of data (observed or forecasted).
    threshold (float): Threshold value for binarizing the data.
    
    Returns:
    np.ndarray: Binary predictor array.
    """
    return (data >= threshold).astype(np.single)

def calculate_np(bp, scale):
    """
    Calculate the Neighborhood Predictor (NP) for the given Binary Predictor (BP) and neighborhood size.
    
    Parameters:
    bp (np.ndarray): Binary predictor array.
    scale (int): Size of the neighborhood for calculating fractions.
    
    Returns:
    np.ndarray: Neighborhood predictor array.
    """
    if scale > 1:
        n_bp = uniform_filter(bp, size=scale, mode='constant', cval=0.0)
    else:
        n_bp = bp
    return n_bp

def calculate_fbs(np_f, np_o):
    """
    Calculate the Fractions Brier Score (FBS).
    
    Parameters:
    np_f (np.ndarray): Neighborhood predictor for the forecasted data.
    np_o (np.ndarray): Neighborhood predictor for the observed data.
    
    Returns:
    float: Fractions Brier Score.
    """
    return np.mean((np_f - np_o) ** 2)

def calculate_wfbs(np_f, np_o):
    """
    Calculate the Weighted Fractions Brier Score (WFBS).
    
    Parameters:
    np_f (np.ndarray): Neighborhood predictor for the forecasted data.
    np_o (np.ndarray): Neighborhood predictor for the observed data.
    
    Returns:
    float: Weighted Fractions Brier Score.
    """
    return np.mean(np_f ** 2) + np.mean(np_o ** 2)

def fss_update(fss, forecast, observed):
    """
    Update the FSS object with new forecast and observed data.
    
    Parameters:
    fss (dict): FSS object.
    forecast (np.ndarray): Forecasted data.
    observed (np.ndarray): Observed data.
    """
    threshold = fss['threshold']
    scale = fss['scale']

    bp_f = calculate_bp(forecast, threshold)
    bp_o = calculate_bp(observed, threshold)
    
    np_f = calculate_np(bp_f, scale)
    np_o = calculate_np(bp_o, scale)

    fss['sum_fct_sq'] += np.sum(np_f ** 2)
    fss['sum_obs_sq'] += np.sum(np_o ** 2)
    fss['sum_fct_obs'] += np.sum(np_f * np_o)

def fss_compute(fss):
    """
    Calculate the Fractions Skill Score (FSS)
    
    Parameters:
    fss (dict): FSS object.
    
    Returns:
    float: Fractions Skill Score.
    """
    sum_fct_sq = fss['sum_fct_sq']
    sum_obs_sq = fss['sum_obs_sq']
    sum_fct_obs = fss['sum_fct_obs']

    fbs = sum_fct_sq + sum_obs_sq - 2 * sum_fct_obs
    wfbs = sum_fct_sq + sum_obs_sq

    if wfbs == 0:
        return np.nan

    fss_value = 1 - fbs / wfbs
    return fss_value


In [6]:

# Create synthetic data
np.random.seed(42)  # For reproducibility
forecast_data = np.random.rand(192, 144)
observed_data = np.random.rand(192, 144)

# Define threshold and scale
threshold = 0.5  # This is a common threshold for binary classification (50th percentile)
scale = 3  # This defines the neighborhood size. Common values are between 1 and 10.

# Initialize the FSS object
fss = fss_init(threshold, scale)

# Update the FSS object with the synthetic data
fss_update(fss, forecast_data, observed_data)

# Compute the FSS value
fss_value = fss_compute(fss)

# Output the result
print(f"Fractions Skill Score (FSS): {fss_value:.4f}")


Fractions Skill Score (FSS): 0.9001
